In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("/home/cat/code/widefield/") # Adds higher directory to python modules path.

import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm
import pandas as pd
import scipy

import os
import time
import warnings
warnings.filterwarnings("ignore")

from locanmf import LocaNMF, postprocess
from locanmf import analysis_fig4 
from visualize import visualize_svm as Visualize
import matplotlib.patches as mpatches
from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax
from scipy import signal


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means


def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                              13,2,
                              12,3,
                              11,4,
                              10,5,
                              9,6,
                              8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    #print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/media/cat/4TBSSD/yuki/yongxu/atlas_split.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    print ("  # trials ", locaNMF_temporal.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random


def plot_locanmf_vs_raw(locaNMF_temporal, raw_means):
# FIg 2B locaNMF vs. raw

    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    raw_means_clip = raw_means[:,:900]
    print ('raw temporal_means: ', raw_means.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,raw_means.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(raw_means_clip.shape[1])/30-30
    fig=plt.figure(figsize=(10,10))
    linewidth=3
    for k in range(raw_means.shape[0]):
        ax1=plt.subplot(121)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.ylim(-.1,1.25)

        temp1 = raw_means_clip[k]#/np.max(raw_means[k])
        if k==0:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth, 
                     label = 'raw')
        else:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth)

        #plt.plot([-15,0], [scale3+k*scale2, scale3+k*scale2],'--',c='black',alpha=.2)


        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k],
                    label='locaNMF')
        else:
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k])

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    #     # 
    #     ax2=plt.subplot(122)
    #     plt.xlim(-15,0)
    #     temp3 = temp2*scale3-temp1*scale1
    #     plt.plot(t, temp3*scale3+k*scale4,'--', c=colors[k])

    ax1.legend()
    if False:
        plt.savefig('/home/cat/fano.png',dpi=300)
        plt.close()
    else:
        plt.show()               

        
        
        
        
        
#
def variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)#[:,:900]
    var = np.array(var)#[:,:900]
    #print (means.shape, var.shape)

    return means, var



def plot_longitudinal_roi_loca(n_trials, saved_names, all_means):
    print ('n trials: ', n_trials)
    colors = plt.cm.viridis(np.linspace(0,1,len(all_means)))
    area_ids = [0,1,6,7,8,9,12,13]
    time= np.arange(all_means[0].shape[1])/30-30.

    #
    ctr=1
    min_trials = 10
    aucs = []
    saved = []
    fig=plt.figure(figsize=(10,6))
    for ctr, area_id in enumerate(area_ids): 
        ax = plt.subplot(2,4,ctr+1)

        # 
        aucs.append([])
        counter = 0
        for t in range(len(all_means)):
            temp = all_means[t][area_id]
            if n_trials[t]<min_trials:
                continue

            if np.max(np.abs(temp))<0.2:
                plt.plot(time, temp, 
                         color=colors[t],
                        alpha=.8)

                auc = np.nansum(np.abs(temp), axis=0)

                aucs[ctr].append([t,auc])
                counter+=1

        print (ctr, 'area_id: ', area_id, counter)
        # 
        #if ctr==5:

        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.title(saved_names[area_id],fontsize=8)

        # cmap = matplotlib.cm.viridis
        #norm = matplotlib.colors.Normalize(vmin=5, vmax=10)

        # cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap,
         #                               orientation='vertical')

    #
    if False:
        plt.savefig('/home/cat/'+str(animal_id)+'_loca_longitudinal.png',dpi=300)
        plt.close()
    else:
        plt.show()
        
        
def plot_locanmf_temporal_averages(fig, 
                                   locaNMF_temporal,
                                  clr):
    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,locaNMF_temporal_means_clip.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(locaNMF_temporal_means_clip.shape[1])/30-30
    linewidth=5
    for k in range(locaNMF_temporal_means_clip.shape[0]):
        #ax1=plt.subplot(121)
        #plt.xticks([])
        #plt.yticks([])
        plt.xlim(-30,0)
        #plt.ylim(-.1,1.25)

        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)
        else:
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)

        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    
    
##################
def plot_variance_locaNMF(fig, var,
                         clr):

    scale1 = 1
    scale2 = .004
    linewidth=5
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    
    
    # 
    for k in range(var.shape[0]):
        # 
        plt.xticks([])
        #plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # plot the variance
        plt.plot(t, temp, c=clr,
                linewidth=linewidth)
        
                
        # 
        #
        print ("temp var: ", temp.shape)
        idx = np.argmin(temp[750:900])
        print ("idx: ", idx)
        print (t[idx+750], temp[idx+750])
        plt.scatter(t[idx+750], temp[idx+750],
                    s=1000,
                    color=clr,
                   alpha=.8)

        # plot lines on top of plot
        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.5)

        # 
        mean2 = np.mean(temp.squeeze())
        plt.plot([-30,0], [mean2,mean2],'--',c=clr,alpha=.5)

    # 
    plt.xlim(-30,0)


def load_locaNMF_temporal(animal_id, session_name, root_dir,
                         session_id):

    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("sessions: ", loca.sessions.shape)
    print ("selected session: ", loca.sessions[session_id])

    session = loca.sessions[session_id]

    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')


    atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random = load_locaNMF_data(fname_locaNMF)

    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random 
    
    
def plot_roi_averages(main_dir,
                     animal_id,
                     session_ids,
                     colors,
                     area_ids,
                     linewidth):
    ctr_session=0

    for session_id in session_ids:
        sessions = Visualize.get_sessions(main_dir,
                         animal_id,
                         session_id)
        #print (sessions)

        loca = np.load(os.path.join(main_dir, animal_id, 'tif_files',sessions[0],
                                    sessions[0]+'_locanmf.npz'))

        trials = loca['temporal_trial']
        random = loca['temporal_random']
        print (session_id, trials.shape[0])
        names = loca['names']
        for k in range(len(names)):
               print (k,names[k])
        # 
        ctr=0
        t=np.arange(900)/30.-30
        for area_id in area_ids:
            plt.subplot(2,2,ctr+1)
            plt.title(names[area_id])

            plt.plot(t,trials[:43,area_id,:900].mean(0),
                      linewidth=linewidth,
                    color=colors[ctr_session])


            ran = random[:43,area_id,:900]

            if True:
                for p in range(ran.shape[0]):
                    idx = np.random.choice(np.arange(-300,300,1))
                    #print (idx)
                    ran[p] = np.roll(ran[p], idx)

            plt.plot(t,ran.mean(0),'--',
                     linewidth=linewidth,
                    color=colors[ctr_session])


            plt.ylim(-0.10, 0.10)
            plt.xlim(-15,t[-1])
            plt.plot([-30,0],[0,0],'--',c='grey')
            ctr+=1

        ctr_session+=1

    if True:
        plt.savefig('/home/cat/osccilations_roi.svg')
        plt.close()
    else:
        plt.show()
        
        
def plot_spectra_longintudinally(colors, plotting, fs, selected_areas, 
                                all_areas,
                                all_means,
                                all_means_random):

    if plotting:
        fig = plt.figure(figsize=(16,8))

    # 
    norm = False
    ctr2=0
    averages = []
    for k in range(len(selected_areas)):
        averages.append([])

    spectra = np.zeros((len(all_means),len(selected_areas),451))
    # loop over all sessions
    for k in range(len(all_means)):

        # get all traces within a session
        all_traces = all_means[k]

        #
        all_traces_random = all_means_random[k]
        
        temp_list = []
        temp_list_random = []
        pxx_list = []
        # loop over all areas selected
        for ctr_area, selected_area in enumerate(selected_areas):

            #########################################
            ######### PLOT INDIVIDUAL SPECGRAMS #####
            #########################################
            temp = all_traces[selected_area]
            temp_random = all_traces_random[selected_area]
            #temp = savgol_filter(temp, 7, 1)
            temp_list.append(temp)
            temp_list_random.append(temp_random)
            if True: #np.max(np.abs(temp))<0.2:
                temp = np.float64(temp)
                temp_random = np.float64(temp_random)
                f, Pxx_den = signal.periodogram(temp, fs)
                f, Pxx_den_random = signal.periodogram(temp_random, fs)

            if norm:
                Pxx_den = Pxx_den/Pxx_den_random

            # SHOW ONLY FIRST 451 datapoints
            x = np.arange(0,451,1)/30.
            Pxx_den = Pxx_den[:451]

            if plotting:
                ax = plt.subplot(1,len(selected_areas),ctr_area+1)
                if norm == False:
                    plt.ylim(1E-8,1E-1)
                else:
                    plt.ylim(1E-3,1E6)

                plt.xlabel('frequency [Hz]')
                plt.xlim(2E-2,6)
                #ax.set_yticks([])
                plt.semilogy()
                plt.semilogx()
                
                plt.plot(x, 
                     Pxx_den,
                     linewidth=3,
                     c= colors[ctr_area][ctr2],
                        alpha=.5)
            # 
            xx = argrelmax(Pxx_den, order=100)

            pxx_list.append(Pxx_den)
            averages[ctr_area].append(Pxx_den)
            
            spectra[k,ctr_area]=Pxx_den
        
        ctr2+=1

    #########################################
    ######### PLOT AVERAGES #################
    #########################################
    for ctr_area in range(len(selected_areas)):
        y = np.median(averages[ctr_area],axis=0)
        argmax = np.argmax(y)

        if plotting:
            ax = plt.subplot(1,len(selected_areas),ctr_area+1)

            plt.plot(x, 
                 y,
                 linewidth=3,
                 c= 'black')

            plt.plot([x[argmax], x[argmax]],[1E-8,1E5], '--',c='red')
            print ("max freq: ", x[argmax])
            #
            plt.plot([0.1, 0.1],[1E-8,1E5], '--', c='black',alpha=.5)
            plt.plot([1, 1],[1E-8,1E5], '--',c='black',alpha=.5)

            for k in range(-8,-1,1):
                plt.plot([1E-4,20],[10**k,10**k], '--',c='black',alpha=.5)


    if plotting:
        plt.savefig('/home/cat/power_spectra_logintiduinal.svg')
        plt.close()
    else:
        plt.show()
        
    return spectra


def plot_box_plots(peaks):
    
    # 
    codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
    #codes = ['limb, layer 1 - right', 'limb, layer 1 - left']
    clrs_local = ['black','blue','red','magenta', 'pink','brown']
    
    # 
    #bin_width = 0.01
    #bins = np.arange(0.1,2.0,bin_width)

    # 
    edts = []
    for a in range(len(peaks)):
        good_vals = np.hstack(peaks[a])
        edts.append(good_vals)
#     # 
    roi_names = ['V1-L',
     'SomF-L',
     'M1-L',
     'RD-L']
    my_dict = dict(V1L = edts[0], 
                   SomFL = edts[1],
                   M1L = edts[2],
                   RDL = edts[3],
#                    M5 = edts[4],                      
#                    M6 = edts[5]
                     )

    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    # 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)

    # manually define a new patch 
    labels = []
    for i,d in enumerate(data):
        y = data[d]#+np.random.uniform(data[d].shape[0])/200.-1/400.
        x = np.random.normal(i+1, 0.04, len(y))

        #
        colors = plt.cm.viridis(np.linspace(0,1,len(edts[i])))
        x = np.random.normal(i+1, 0.04, len(edts[i]))
        print (i,d, ' y shape: ', y.shape)
        plt.scatter(x, edts[i], 
                   #c=clrs_local[i],
                   c=colors,
                   edgecolor='black',
                   s=200,
                   #alpha=np.linspace(.2, 1.0, x.shape[0])
                   alpha=.5
                   )
        
        # compute correlation between time and location
        from scipy import stats
        # Y and Z are numpy arrays or lists of variables 
        #print (np.arange(edts[i]).shape, y.shape)
        corr = stats.pearsonr(np.arange(y.shape[0]), y)
        #res = scipy.stats.normaltest(edts[i])
        print (i, "corr", corr)
        
        # 
        patch = mpatches.Patch(color='grey', label=roi_names[i]+ ":  "+str(round(corr[0],2)))

        # handles is a list, so append manual patch
        labels.append(patch) 

        # plot the legend
    plt.legend(handles=labels, loc='upper center')

    
    
        
    plt.xticks([])
    plt.yticks([])
    
    plt.semilogy()
    #plt.ylim(1E-1,1E1)
    

In [9]:
#######################################################
########### LOAD LIMB AREAS AND TIME COURSES ##########
#######################################################

# FIG 6A example of visual vs. limb averages

main_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IJ2'
session_ids = ['Feb1_', 'Mar2_','Mar31_','Apr4_']
session_ids = ['Feb3_','Mar2_'] #,'Apr4_']

colors = plt.cm.viridis(np.linspace(0,1,3))
#colors=['black','blue','red','magenta']
# 
plt.figure(figsize=(15,10))
area_ids = [13, 8, 12, 11]
linewidth = 3
plot_roi_averages(main_dir,
                     animal_id,
                     session_ids,
                     colors,
                     area_ids,
                     linewidth)




Feb3_ 43
0 Retrosplenial area, dorsal part, layer 1 - right
1 Retrosplenial area, lateral agranular part, layer 1 - right
2 Primary visual area, layer 1 - right
3 Primary somatosensory area, upper limb, layer 1 - right
4 Primary somatosensory area, lower limb, layer 1 - right
5 Primary somatosensory area, barrel field, layer 1 - right
6 Secondary motor area, layer 1 - right
7 Primary motor area, Layer 1 - right
8 Primary motor area, Layer 1 - left
9 Secondary motor area, layer 1 - left
10 Primary somatosensory area, barrel field, layer 1 - left
11 Primary somatosensory area, lower limb, layer 1 - left
12 Primary somatosensory area, upper limb, layer 1 - left
13 Primary visual area, layer 1 - left
14 Retrosplenial area, lateral agranular part, layer 1 - left
15 Retrosplenial area, dorsal part, layer 1 - left
Mar2_ 106
0 Retrosplenial area, dorsal part, layer 1 - right
1 Retrosplenial area, lateral agranular part, layer 1 - right
2 Primary visual area, layer 1 - right
3 Primary somatosen

In [10]:
####################################################################
###### LOAD MEAN AND VARIANCE FOR EACH SESSION ROI ACTIVITY ########
####################################################################
#
animal_id = 'IJ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

# 
loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

#
loca.get_sessions(session_name)
print ("# of sessions: ", loca.sessions.shape)

all_means = []
n_trials = []
saved_names = []
all_means_random = []
good_sessions = []
all_loca_NMF = []
for session in tqdm(loca.sessions):
    
    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')
    atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

    # 
    if atlas is not None:
        means, var = variance_locaNMF(locaNMF_temporal)
        n_trials.append(locaNMF_temporal.shape[0])
        all_means.append(means)
        
        means, var = variance_locaNMF(random)
        all_means_random.append(means)
                
        saved_names = names
        good_sessions.append(session)
    
print ("# good sessions: ", len(good_sessions))

  7%|▋         | 3/44 [00:00<00:02, 19.99it/s]

# of sessions:  (44,)
  # trials  (10, 16, 1801)
  # trials  (16, 16, 1801)
  # trials  (43, 16, 1801)


 11%|█▏        | 5/44 [00:00<00:03, 11.30it/s]

  # trials  (47, 16, 1801)
  # trials  (52, 16, 1801)
file missing /media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb9_30Hz/IJ2pm_Feb9_30Hz_locanmf.npz


 20%|██        | 9/44 [00:00<00:02, 11.98it/s]

...

  # trials  (33, 16, 1801)
  # trials  (65, 16, 1801)


 95%|█████████▌| 42/44 [00:04<00:00,  6.99it/s]

  # trials  (82, 16, 1801)
  # trials  (58, 16, 1801)


100%|██████████| 44/44 [00:04<00:00,  9.52it/s]

  # trials  (37, 16, 1801)
  # trials  (60, 16, 1801)
# good sessions:  39


In [12]:
########################################
###### PLOT LONGITUDINAL RESULTS #######
#########################################
# # fig 5F longintudinal power spectra 
# plot_longitudinal_roi_loca(n_trials, saved_names, all_means)   

In [16]:
##############################
####### POWER SPECTRUM #######
##############################

# Fig 7b power vs. freq spectra for major regions
colors = []
colors.append(plt.cm.viridis(np.linspace(0,1,len(all_means))))
colors.append(plt.cm.viridis(np.linspace(0,1,len(all_means))))
colors.append(plt.cm.viridis(np.linspace(0,1,len(all_means))))
#colors.append(plt.cm.Blues(np.linspace(0,1,len(all_means))))

# 
fs = 30
plotting=True
selected_areas = [11,9,1,15]
print (saved_names[selected_areas])
all_areas = np.arange(all_means[0].shape[0])

#
spectra = plot_spectra_longintudinally(colors, plotting, 
                                       fs, 
                                       selected_areas, 
                                       all_areas,
                                       all_means,
                                       all_means_random)

#


['Primary visual area, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary motor area, Layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']
max freq:  0.4666666666666667
max freq:  0.4666666666666667
max freq:  0.4666666666666667
max freq:  0.5


In [252]:
##################################################
###### PEAK POWER AND PEAK FREQUENCY PLOTS #######
##################################################

session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

plot_peak_frequency = True
plot_peak_power = False
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']#,'IA2','IA3','IJ1','IJ2','AQ2']


all_peaks = []
#all_power = []
for animal_id in animal_ids:

    ##############################################
    ########### GET LOCA COMPONENTS ##############
    ##############################################
    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    all_means = []
    n_trials = []
    saved_names = []
    all_means_random = []
    good_sessions = []
    for session in tqdm(loca.sessions):

        # load data
        fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                     session + '_locanmf.npz')
        atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

        # 
        if atlas is not None:
            means, var = variance_locaNMF(locaNMF_temporal)
            n_trials.append(locaNMF_temporal.shape[0])
            all_means.append(means)

            means, var = variance_locaNMF(random)
            all_means_random.append(means)

            saved_names = names
            good_sessions.append(session)
    
    print ("# good sessions: ", len(good_sessions))
        
    ##############################################
    ########### GET SPECTRA ######################
    ##############################################
    fs = 30
    plotting=False
    selected_areas = [11,9,1,15]
    print (saved_names[selected_areas])
    all_areas = np.arange(all_means[0].shape[0])

    #
    spectra = plot_spectra_longintudinally(colors, plotting, fs, selected_areas, 
                                all_areas,
                                all_means,
                                all_means_random)

    #####################################
    ########### PLOT POWER ##############
    #####################################
    print (spectra.shape)
    x = np.arange(0,451,1)/30.

    peaks = []
    for a in range(spectra.shape[1]):
        peaks.append([])
        for s in range(spectra.shape[0]):
            temp = spectra[s,a]
            argmax = np.argmax(temp)

            # plot peak frequency
            if plot_peak_frequency:
                peaks[a].append(x[argmax])
            # plot frequency power
            elif plot_peak_power:
                peaks[a].append(temp[argmax])

    ############
    fig = plt.figure(figsize=(15,10))
    plot_box_plots(peaks)
    all_peaks.append(peaks)
    #all_power.append(peaks)

    if False:
        plt.savefig('/home/cat/'+animal_id+'_peakpower.svg')
        plt.close()
    else:
        plt.show()

  6%|▌         | 4/71 [00:00<00:01, 39.79it/s]

# of sessions:  (71,)
file missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_locanmf.npz
  # trials  (23, 16, 1801)
  # trials  (58, 16, 1801)
file missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz_locanmf.npz
  # trials  (51, 16, 1801)
  # trials  (13, 16, 1801)


 17%|█▋        | 12/71 [00:00<00:01, 33.05it/s]

  # trials  (38, 16, 1801)
file missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz_locanmf.npz
  # trials  (18, 16, 1801)
  # trials  (40, 16, 1801)
  # trials  (36, 16, 1801)
  # trials  (14, 16, 1801)


 23%|██▎       | 16/71 [00:00<00:02, 26.99it/s]

...

  # trials  (114, 16, 1801)
  # trials  (112, 16, 1801)


 99%|█████████▉| 109/110 [00:07<00:00,  9.47it/s]

  # trials  (138, 16, 1801)
  # trials  (86, 16, 1801)
  # trials  (82, 16, 1801)


100%|██████████| 110/110 [00:07<00:00, 15.39it/s]


# good sessions:  77
['Primary visual area, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary motor area, Layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']
(77, 4, 451)
0 V1L  y shape:  (77,)
0 corr (0.19705950898847968, 0.08583406455828445)
1 SomFL  y shape:  (77,)
1 corr (0.11737700598252553, 0.3093190790767045)
2 M1L  y shape:  (77,)
2 corr (-0.11510223939037657, 0.31885398273169263)
3 RDL  y shape:  (77,)
3 corr (0.11999855653007879, 0.2985620215174832)


In [259]:
from sklearn.linear_model import LinearRegression
from scipy import stats

#########################################
ids = [0,2,3]
#feats = ['motor','visual','Retro','limb','barrel']
clrs_local = ['magenta','brown','pink','lightblue','darkblue']

feats_ = ['visual','limb','motor','Retro']
clrs_local_ = ['brown','lightblue','magenta','pink']

ids_ = np.array([1,2,3])
# 
animal_id = 0
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']#,'IA2','IA3','IJ1','IJ2','AQ2']

# 
fig=plt.figure(figsize=(10,10))
for ctr_animal, animal_id in enumerate(animal_ids):
    #maxes = all_peaks[ctr_animal]
    maxes = all_power[ctr_animal]
    print ("maxes: ", maxes)
    
    ax=plt.subplot(3,2,ctr_animal+1)

    ctr=0
    for id_ in ids:
        m = np.array(maxes[id_])

        ##################################
        ##################################
        ##################################
        corr = stats.pearsonr(np.arange(m.shape[0]), m)


        model = LinearRegression()
        y=np.array(m).reshape(-1, 1)
        x = np.arange(y.shape[0]).reshape(-1, 1)
        model.fit(x, y)

        x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
        y_pred = model.intercept_ + model.coef_ * x2


        # compute correlation between time and location

        print (i, "corr", corr)

        plt.plot(x2, y_pred, 
                 label= str(np.round(corr,2)),
                  c=clrs_local[id_],
                 linewidth=6)
        # 
        #patch = mpatches.Patch(color='grey', label=feats[id_]+ ":  "+str(round(corr[0],2)))

        # handles is a list, so append manual patch
        #labels.append(patch) 

        ##################################
        ##################################
        ##################################
        # 
        plt.scatter(np.arange(m.shape[0]), m,
                   c=clrs_local[id_],
                    edgecolor='black',
                    s=100,
                   alpha=.3)

        #
        #plt.title(feats[id_])
        #ax.set_ylim(0, 1)
        ax.set_ylim(0, 0.005)
        ax.set_xlim(0,x[-1])
        ctr+=1
        plt.legend(fontsize=14)
    
if True:
    #plt.savefig('/home/cat/fit_trends_psd_averages_frequency.svg')
    plt.savefig('/home/cat/fit_trends_psd_averages_power.svg')
    plt.close()
else:
    plt.show()



maxes:  [[0.0006116605409541181, 0.0018849881352795087, 0.0005712364193017595, 0.000401812417373834, 0.0010395803007610047, 0.00024643180727060244, 0.0005871480034561305, 0.0003025191250931239, 0.0008247617446347441, 0.00012822770809420002, 7.729696128631388e-05, 0.0005464615003800622, 0.0007782033159817657, 0.0002280681255515222, 0.00024759539987041746, 5.722802130593188e-05, 0.00043647557710074395, 0.0028136917769286548, 0.0007997780021914752, 0.0002417434475609279, 0.0006413137822402282, 0.0003273375253157739, 0.0004554303246891655, 0.00027565712146280705, 0.0006350569095176117, 0.0006307913041931154, 0.000472176673019456, 0.0001552767360869647, 0.0003535815961375419, 0.00015668089052798645, 0.00033245526966217226, 0.00024654775413950476, 0.0004531260621376245, 0.00021128605800226348, 0.00023014932638307375, 0.0001819866823574315, 0.00020054457725730826, 0.0002931476107421512, 6.504229318180982e-05, 8.988246016809024e-05, 8.326117841756405e-05, 0.00011559903356172457, 0.000214313674

0 corr (-0.11392496941430799, 0.48983771173716756)
0 corr (0.10481198634522296, 0.5254236496869793)
maxes:  [[0.0012450651167520434, 4.50077057866075e-05, 0.00011764998861478799, 8.312452140319685e-05, 0.0002512788626979311, 7.310309202410571e-05, 0.0006267069491795505, 0.00018622904244298906, 9.341833113428417e-05, 0.0001583790580003453, 0.0003443836292758801, 0.00034816125849617324, 8.176687007054361e-05, 0.0005862820903904639, 0.000258190272535307, 0.0009519607185139826, 0.0009355148656504005, 0.0024683451108644364, 3.47381514425594e-05, 5.264642615869488e-05, 3.347937045794994e-05, 7.464804014935505e-05, 7.136631054409717e-05, 0.0001334396808345932, 0.00011418829247813653, 0.0002801292891489333, 0.0001492285200046042, 6.445406992001873e-05, 0.00023419533534522663, 0.0002486108720363033, 0.000721444593088815, 0.00036385785596823345, 0.0006880596624471963, 0.00033797805686919995, 0.00045394628327471385, 0.0007659529322058648, 0.00024227630132567324, 0.0003439955093834235, 0.000181458

### HERE we repeat above functions but for individaul sessions


In [184]:
####################################################################
###### LOAD MEAN AND VARIANCE FOR EACH SESSION ROI ACTIVITY ########
####################################################################
#
from scipy import signal
def compute_psd(data):
    print (data.shape)
    
    fs = 30
    rois = []
    for roi in range(data.shape[1]):
        
        psd = []
        for t in range(data.shape[0]):
            x=data[t,roi]
            f, Pxx_den = signal.periodogram(x, fs)
            psd.append(Pxx_den)
            
        psd=np.vstack(psd)
        psd = np.median(psd,axis=0)
        rois.append(psd)
        
    return rois, f

def compute_psd_whole_stack(data):
    print (data.shape)
    
    fs = 30
    rois = []
    for roi in range(data.shape[0]):
        x = data[roi]
        f, Pxx_den = signal.periodogram(x, fs)
        rois.append(Pxx_den)
                  
    return rois, f

def load_locaNMF_wholestack(root_dir,
                           animal_id,
                           session):
    fname = os.path.join(root_dir,
                                   animal_id,
                         'tif_files',
                                    session,
                                    session+'_locanmf_wholestack.npz')
    print (fname)
    try:
        data = np.load(fname, 
                       allow_pickle=True)

        trials = data['whole_stack']
        print (trials.shape)

        return trials
    except:
        return None

###########################################



session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

animal_id = 'AQ2'
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

for animal_id in animal_ids:
    # 
    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    all_means = []
    n_trials = []
    saved_names = []
    all_means_random = []
    good_sessions = []
    all_loca_NMF = []
    psds = []

    # 
    psds_random = []

    t1 = 0
    t2 = 1800
    
    fname_out = '/home/cat/psds_'+str(animal_id)+'.npy'
    if os.path.exists(fname_out)==False:

        for session in tqdm(loca.sessions):

            # load data
            if True:
                fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                             session + '_locanmf.npz')
                atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

                #  
                if atlas is not None:

                    psd, f = compute_psd(locaNMF_temporal[:,:,t1:t2])
                    psds.append(psd)
                    psd, f = compute_psd(random[:,:,t1:t2])
                    psds_random.append(psd)

                    #
                    n_trials.append(locaNMF_temporal.shape[0])
                    saved_names = names
                    good_sessions.append(session)


            else:
                trials = load_locaNMF_wholestack(root_dir,
                                               animal_id,
                                               session)

                if trials is not None:
                    psd, f = compute_psd_whole_stack(trials)
                    psds.append(psd)




            ctr+=1


        print ("# good sessions: ", len(good_sessions))
        
        
        np.save(fname_out, psds)

  0%|          | 0/71 [00:00<?, ?it/s]

# of sessions:  (71,)
file missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_locanmf.npz
  # trials  (23, 16, 1801)
(23, 16, 1800)
(23, 16, 1800)


  3%|▎         | 2/71 [00:00<00:08,  7.81it/s]

  # trials  (58, 16, 1801)
(58, 16, 1800)
(58, 16, 1800)


  4%|▍         | 3/71 [00:00<00:22,  3.00it/s]

...

  # trials  (86, 16, 1801)
(86, 16, 1800)
(86, 16, 1800)


 99%|█████████▉| 109/110 [01:06<00:01,  1.07s/it]

  # trials  (82, 16, 1801)
(82, 16, 1800)
(82, 16, 1800)


100%|██████████| 110/110 [01:07<00:00,  1.62it/s]

# good sessions:  77


In [188]:
####################
#ROI_names = saved_names.copy()

feats = ['motor','visual','Retro','limb','barrel']

plotting = False
all_maxes = []
for animal_id in animal_ids:

    if plotting:
        fig=plt.figure()

    fname_out = '/home/cat/psds_'+str(animal_id)+'.npy'

    psds = np.load(fname_out, allow_pickle=True)
    #cmap = matplotlib.cm.get_cmap('Paired', len(psds[session_id]))   
    cmap = matplotlib.cm.get_cmap('jet', len(psds))   

    sessions = np.arange(len(psds))
    #sessions = np.arange(0,len,5)
    print ("# sessions: ", len(sessions))
    ctr=0
    maxes = []
    for feat in feats:
        maxes.append([])
        if plotting:
            ax=plt.subplot(2,3,ctr+1)
            plt.title(feat)
        for session_id in sessions:
            for k in range(len(psds[session_id])):
                if feat in ROI_names[k]:
                    temp = psds[session_id][k]
                    #temp = psds_random[session_id][k]
                    if True:
                        temp = scipy.signal.savgol_filter(temp, 
                                                          window_length=5, 
                                                          polyorder=1, 
        #                                                   deriv=0, 
        #                                                   delta=1.0, 
        #                                                   axis=- 1, 
        #                                                   mode='interp', 
        #                                                   cval=0.0
                                                         )

                    argmax = np.argmax(temp)
                    maxes[ctr].append([f[argmax],temp[argmax]])

                    #
                    if plotting:
                        plt.scatter(f[argmax],temp[argmax],
                                   c='black',
                                   s=100)
                        plt.plot(f,temp,
                             c=cmap(session_id),
                             #label=names[k]
                                 alpha=.25
                                )


            #plt.legend()   
            if plotting:
                plt.semilogx()
                plt.semilogy()
                #plt.xlim(0,6)
                #plt.ylim(1E-4,1E6)
                plt.ylim(1E-5,1E-1)
        ctr+=1
    all_maxes.append(maxes)
if plotting:        
    plt.show()

# sessions:  53
# sessions:  31
# sessions:  41
# sessions:  40
# sessions:  39
# sessions:  77


In [263]:
from sklearn.linear_model import LinearRegression
from scipy import stats

#####
ids = [0,2,3]
feats = ['motor','visual','Retro','limb','barrel']
clrs_local = ['magenta','brown','pink','lightblue','darkblue']

# 
animal_id = 5

# 
fig=plt.figure(figsize=(10,10))
for ctr_animal, animal_id in enumerate(animal_ids):
    maxes = all_maxes[ctr_animal]
    ax=plt.subplot(3,2,ctr_animal+1)

    ctr=0
    for id_ in ids:
    #    ax=plt.subplot(3,1,ctr+1)
        m = np.array(maxes[id_])[:,1]    #<--------------- 0: freq; 1: power

        ##################################
        ##################################
        ##################################
        corr = stats.pearsonr(np.arange(m.shape[0]), m)


        model = LinearRegression()
        y=np.array(m).reshape(-1, 1)
        x = np.arange(y.shape[0]).reshape(-1, 1)
        model.fit(x, y)

        x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
        y_pred = model.intercept_ + model.coef_ * x2


        # compute correlation between time and location

        print (i, "corr", corr)

        plt.plot(x2, y_pred, 
                 label= str(np.round(corr,2)),
                  c=clrs_local[id_],
                 linewidth=6)
        # 
        #patch = mpatches.Patch(color='grey', label=feats[id_]+ ":  "+str(round(corr[0],2)))

        # handles is a list, so append manual patch
        #labels.append(patch) 

        ##################################
        ##################################
        ##################################
        # 
        plt.scatter(np.arange(m.shape[0]), m,
                   c=clrs_local[id_],
                    edgecolor='black',
                    s=100,
                   alpha=.3)

        #
        #plt.title(feats[id_])
        ax.set_ylim(bottom=0) #0, 0.7)
        #ax.set_ylim(0, 0.04)
        ax.set_xlim(0,x[-1])
        ctr+=1
        plt.legend(fontsize=14)
    
if True:
    plt.savefig('/home/cat/fit_trends_psd_single_trials_power.svg')
    plt.close()
else:
    plt.show()





0 corr (-0.37517865981880355, 1.7235703434788164e-08)
0 corr (-0.34361018930213916, 2.889225272905296e-07)
0 corr (-0.6357704963380455, 2.0858373104983265e-25)
0 corr (-0.09208872471412914, 0.3090418875392825)
0 corr (-0.40892638304908896, 2.4130264367503185e-06)
0 corr (-0.13255413136391697, 0.1422141189714407)
0 corr (0.03872690419744708, 0.6224806215021097)
0 corr (0.034853583323691525, 0.6577192206151463)
0 corr (0.16410258507734138, 0.035753449719757587)
0 corr (0.3247245546820134, 2.7972739927185304e-05)
0 corr (0.2993308058810632, 0.00012049183670800633)
0 corr (0.3974492362791603, 1.9509681288218603e-07)
0 corr (0.10110506661965864, 0.20916436286224335)
0 corr (0.10703432144719635, 0.18354120068016902)
0 corr (0.04855338053624437, 0.5472344145918647)
0 corr (-0.06880038618211162, 0.22860524789995587)
0 corr (-0.09445482279588213, 0.0979982674205848)
0 corr (0.23193914053156106, 3.9540184959808677e-05)


In [156]:
feats = ['motor','visual','Retro','limb','barrel']
print (ROI_names)
maxes = np.array(maxes)
print (maxes.shape)
print (np.vstack(maxes[4]).shape)
my_dict = dict(motor = np.array(maxes[0])[:,0], 
               visual = np.array(maxes[1])[:,0],
               retro = np.array(maxes[2])[:,0],
               limb = np.array(maxes[3])[:,0],
               barrel = np.array(maxes[4])[:,0]
#                    M5 = edts[4],                      
#                    M6 = edts[5]
                 )

data = pd.DataFrame.from_dict(my_dict, orient='index')
data = data.transpose()

# 
flierprops = dict(marker='o', 
                  #markerfacecolor='g', 
                  #markersize=10000,
                  linestyle='none', 
                  markeredgecolor='r')

#
data.boxplot(showfliers=False,
            flierprops=flierprops)

# manually define a new patch 
labels = []
for i,d in enumerate(data):
    y = data[d]#+np.random.uniform(data[d].shape[0])/200.-1/400.
    x = np.random.normal(i+1, 0.04, len(y))

    #
    colors = plt.cm.viridis(np.linspace(0,1,len(maxes[i])))
    x = np.random.normal(i+1, 0.04, len(maxes[i]))
    print (i,d, ' y shape: ', y.shape)
    plt.scatter(x, np.array(maxes[i])[:,0], 
               #c=clrs_local[i],
               c=colors,
               edgecolor='black',
               s=200,
               #alpha=np.linspace(.2, 1.0, x.shape[0])
               alpha=.5
               )

    # compute correlation between time and location
    from scipy import stats
    # Y and Z are numpy arrays or lists of variables 
    #print (np.arange(edts[i]).shape, y.shape)
    corr = stats.pearsonr(np.arange(y.shape[0]), y)
    #res = scipy.stats.normaltest(edts[i])
    print (i, "corr", corr)

    # 
    patch = mpatches.Patch(color='grey', label=roi_names[i]+ ":  "+str(round(corr[0],2)))

    # handles is a list, so append manual patch
    labels.append(patch) 

    # plot the legend
plt.legend(handles=labels, loc='upper center')

plt.show()




['Primary motor area, Layer 1 - right'
 'Primary motor area, Layer 1 - left'
 'Secondary motor area, layer 1 - right'
 'Secondary motor area, layer 1 - left'
 'Primary somatosensory area, barrel field, layer 1 - right'
 'Primary somatosensory area, barrel field, layer 1 - left'
 'Primary somatosensory area, lower limb, layer 1 - right'
 'Primary somatosensory area, lower limb, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - right'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary visual area, layer 1 - right'
 'Primary visual area, layer 1 - left'
 'Retrosplenial area, lateral agranular part, layer 1 - right'
 'Retrosplenial area, lateral agranular part, layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - right'
 'Retrosplenial area, dorsal part, layer 1 - left']
(5,)
(106, 2)
0 motor  y shape:  (212,)
0 corr (0.6292560293255796, 8.914053845499193e-25)


NameError: name 'roi_names' is not defined

In [97]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_Mar4_30Hz/IA1am_Mar4_30Hz_locanmf_wholestack.npz')
trials = data['whole_stack']
print (trials.shape)

(16, 40000)


In [50]:
a = np.array([[0,1]])
plt.figure(figsize=(1.5,9))
img = plt.imshow(a, cmap="viridis")
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
plt.colorbar(orientation="vertical", cax=cax)
plt.savefig("/home/cat/colorbar.svg")
plt.close()

In [105]:
###############################
###### VARIANCE ANALYSIS ######
###############################


animal_id = 'AQ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'


# 
loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

#
loca.get_sessions(session_name)
print ("# of sessions: ", loca.sessions.shape)

session_id = 15
session = loca.sessions[session_id]
    
# load data
fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                             session + '_locanmf.npz')

print ("fname: ", fname_locaNMF)
atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

# 
if atlas is not None:
    means, var = variance_locaNMF(locaNMF_temporal)
    n_trials.append(locaNMF_temporal.shape[0])
    all_means.append(means)

    means, var = variance_locaNMF(locaNMF_temporal)
    all_means_random.append(means)

    saved_names = names
    
    fig = plt.figure()
    plot_variance_locaNMF(fig, var)
    
else:
    print ('session not good')

# of sessions:  (110,)
fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec29_30Hz/AQ2am_Dec29_30Hz_locanmf.npz
  # trials  (33, 16, 1801)


In [ ]:
###################################################
############ PLOT MEANS AND VARIANCES #############
###################################################
# 
animal_id = 'IJ1'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

session_ids = [0,17,25,42]

colors = plt.cm.viridis(np.linspace(0,1,len(session_ids)))

random_flag = False
################################################ 
fig=plt.figure(figsize=(10,10))
for ctr,session_id in enumerate(session_ids):
    print 

    loca = load_locaNMF_temporal(animal_id, 
                                  session_name, 
                                  root_dir,
                                  session_id)

    # if using random data
    if random_flag:
        loca.locaNMF_temporal = loca.locaNMF_temporal_random
        
    ###################################################
    if atlas is not None:
        #print ("areas: ", areas)
        #print ("names: ", names)

        area_sel = 'upper limb, layer 1 - left'
        areas_selected = []
        for k in range(len(names)):
            #print ("areas", names[k])
            if area_sel in loca.names[k]:
                areas_selected.append(k)

        areas_selected = np.array(areas_selected)
        locaNMF_temporal = locaNMF_temporal[:,areas_selected]

        ##########################################

        ax=plt.subplot(2,1,1)
        clr = colors[ctr]

        plot_locanmf_temporal_averages(fig, 
                                       locaNMF_temporal,
                                      clr)
        plt.xlim(-15,0)


        ####################################################
        means, var = variance_locaNMF(locaNMF_temporal)
        all_means_random.append(means)

        saved_names = names
        ax=plt.subplot(2,1,2)
        plot_variance_locaNMF(fig, var,
                             clr)
        plt.xlim(-15,0)

        #plt.show()

if True:
    plt.savefig('/home/cat/means_vars.svg')
    plt.close()
else:
    plt.show()

In [37]:

data = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3am_Mar15_30Hz/IA3am_Mar15_30Hz_locanmf.npz',
              allow_pickle=True)

A_reshape = data['A_reshape']
temp = data['temporal_trial']
names = data['areanames_area']

print (temp.shape)
mean = temp.mean(0)


fig=plt.figure()
for i in range(A_reshape.shape[2]):
    plt.subplot(4,4,i+1)
    plt.imshow(A_reshape[:,:,i])
    plt.xticks([])
    plt.yticks([])
    plt.title(names[i],fontsize=6)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.show()




(34, 16, 1801)


In [24]:

for k in range(mean.shape[0]):
    plt.plot(mean[k,:900]+k*.1)
plt.show()

(11, 16, 1801)
